# Projektarbeit Gruppe 2: Gesundheitsdaten

https://www.kaggle.com/datasets/sooyoungher/smoking-drinking-dataset

In [19]:
# Import der benötigten Bibliotheken

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm

from statsmodels.formula.api import ols

from statsmodels.stats.anova import anova_lm

In [10]:
# Neuer Datensatz
path = str(os.getcwd()).replace("\Projektarbeit\Aufgaben", "")

# Lesen der CSV Datei:
dataset = pd.read_csv(path + '\dataset_custom.csv')

# Trennung in des Dataset in Frauen und Männer
data_m = dataset[dataset['Geschlecht'] == 0]

data_f = dataset[dataset['Geschlecht'] == 1]

In [7]:
# Um die Sample zahl zu erhöhren werden die alle BD-Spalten zusammen gefasst.
#ANstelle der feinen unterscheidung die simplere darstellung um hoffentlich bessere Ergebnisse zu bekommen.

def summerize_col(data, neueSpalte, Spalte1, Spalte2, Spalte3):
        data[neueSpalte] = data[[Spalte1, Spalte2, Spalte3]].any(axis=1).astype(int)
        data = data.drop(columns=[Spalte1, Spalte2, Spalte3])
        return data

In [12]:
data_m = summerize_col(data_m, 'Hypertonie', 'BD_Hypertonie Grad 1', 'BD_Hypertonie Grad 2', 'BD_Hypertonie Grad 3')
data_f = summerize_col(data_f, 'Hypertonie', 'BD_Hypertonie Grad 1', 'BD_Hypertonie Grad 2', 'BD_Hypertonie Grad 3')

data_m = summerize_col(data_m, 'BD_unkritisch', 'BD_optimal', 'BD_normal', 'BD_Grenzwert_normal')
data_f = summerize_col(data_f, 'BD_unkritisch', 'BD_optimal', 'BD_normal', 'BD_Grenzwert_normal')

C:\Users\imanu\AppData\Local\Temp\ipykernel_27184\2465580812.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[neueSpalte] = data[[Spalte1, Spalte2, Spalte3]].any(axis=1).astype(int)
C:\Users\imanu\AppData\Local\Temp\ipykernel_27184\2465580812.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[neueSpalte] = data[[Spalte1, Spalte2, Spalte3]].any(axis=1).astype(int)


In [13]:
data_m.head()

,Unnamed: 0,Geschlecht,Alter,Größe,Gewicht,Hüftumfang,Nüchterner Blutzucker,Totale Cholesterin,HDL_Cholesterin,LDL_Cholesterin,...,SGOT_AST,SGOT_ALT,gamma_GTP,Body-Mass-Index,Raucher_nie,Raucher_ehem,Raucher_aktiv,Trinker,Hypertonie,BD_unkritisch
0,0,0,35,170,75,90.0,99.0,193.0,48.0,126.0,...,21.0,35.0,40.0,25.95,1,0,0,1,0,1
1,1,0,30,180,80,89.0,106.0,228.0,55.0,148.0,...,20.0,36.0,27.0,24.69,0,0,1,0,0,1
2,2,0,40,165,75,91.0,98.0,136.0,41.0,74.0,...,47.0,32.0,68.0,27.55,1,0,0,0,0,1
3,3,0,50,175,80,91.0,95.0,201.0,76.0,104.0,...,29.0,34.0,18.0,26.12,1,0,0,0,1,0
4,4,0,50,165,60,80.0,101.0,199.0,61.0,117.0,...,19.0,12.0,25.0,22.04,1,0,0,0,0,1


# Anforderungen 8: Regression

Entscheiden Sie, ob Sie mit der linearen Regression modellieren, prädizieren, oder beides erreichen wollen.

Vorerst modellieren, da prädizieren schon mit der Softmax Regression nicht befriedigend war. 
Evtl. können durch modellieren weitere Erkenntnisse gewonnen werden. Erfreulich wäre so etwas wie: "Personen mit den Merkmalen X neigen dazu an Hypertonie zu leiden."

Führen Sie die entsprechende(n) Analyse(n) methodisch sauber durch und berichten Sie entsprechend der    eingeführten Kriterien und Evaluationsmaße das Ergebnis.

## @Sebi und Josh: Die im folgenden beschriebenen Korrelationen errechenn sich aus der Wurzel des "R-squared"-Wertes (oben rechts beim Output des summarys). Siehe auch Skript meine Mäuse :*

In [22]:
# Syntax für die Regressionsgleichung: Zielwert ~ Feature 1 + Feature 2 + ...
hypertonie_1= ols("Hypertonie ~ HDL_Cholesterin + LDL_Cholesterin + Trinker + Raucher_aktiv + Raucher_ehem + Hämoglobin", data_m).fit()

print(hypertonie_1.summary())

                            OLS Regression Results                            
Dep. Variable:             Hypertonie   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     527.8
Date:                Fri, 08 Dec 2023   Prob (F-statistic):               0.00
Time:                        17:21:06   Log-Likelihood:            -2.0750e+05
No. Observations:              526415   AIC:                         4.150e+05
Df Residuals:                  526408   BIC:                         4.151e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.1005      0.007    -

### Erstes Ergebnis bei Männer nur 7% Korrelation

In [24]:
# Syntax für die Regressionsgleichung: Zielwert ~ Feature 1 + Feature 2 + ...
hypertonie_1f= ols("Hypertonie ~ HDL_Cholesterin + LDL_Cholesterin + Trinker + Raucher_aktiv + Raucher_ehem + Hämoglobin", data_f).fit()

print(hypertonie_1f.summary())

                            OLS Regression Results                            
Dep. Variable:             Hypertonie   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     833.3
Date:                Fri, 08 Dec 2023   Prob (F-statistic):               0.00
Time:                        17:21:32   Log-Likelihood:                -99323.
No. Observations:              464874   AIC:                         1.987e+05
Df Residuals:                  464867   BIC:                         1.987e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0835      0.005    -

### Zweites Ergebnis bei Frauen 10% Korrelation. Trotz gleicher PAramter höhere Korrelation

In [28]:
hypertonie_2m= ols("Hypertonie ~ Triglycerid + Alter + SGOT_AST + SGOT_ALT + Größe + Gewicht + Hüftumfang  + HDL_Cholesterin + LDL_Cholesterin + Trinker + Raucher_aktiv + Raucher_ehem + Hämoglobin", data_m).fit()

print(hypertonie_2m.summary())

                            OLS Regression Results                            
Dep. Variable:             Hypertonie   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     2264.
Date:                Fri, 08 Dec 2023   Prob (F-statistic):               0.00
Time:                        17:26:08   Log-Likelihood:            -1.9476e+05
No. Observations:              526415   AIC:                         3.896e+05
Df Residuals:                  526401   BIC:                         3.897e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.1886      0.019     

### Mit allen Paramter eine 23% Korrelation

In [29]:
hypertonie_2f= ols("Hypertonie ~ Triglycerid + Alter + SGOT_AST + SGOT_ALT + Größe + Gewicht + Hüftumfang  + HDL_Cholesterin + LDL_Cholesterin + Trinker + Raucher_aktiv + Raucher_ehem + Hämoglobin", data_f).fit()

print(hypertonie_2f.summary())

                            OLS Regression Results                            
Dep. Variable:             Hypertonie   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     2588.
Date:                Fri, 08 Dec 2023   Prob (F-statistic):               0.00
Time:                        17:27:06   Log-Likelihood:                -85568.
No. Observations:              464874   AIC:                         1.712e+05
Df Residuals:                  464860   BIC:                         1.713e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0866      0.016     

### Mit allen Paramter  eine 25% Korrelation für Frauen.

Interessanter weise ist keines der einzelnen Paramter für sich relveant. (abzulesen an der Spalte "coef", unterer Teil des Summary-outputs).
Vllt könnt ihr mal Frau Pado fragen warum und was das bedeutet. 
Da kein Paramter besonderns heraussticht, weiß ich vorerst nicht so richtig wie ich weiter machen soll oder wie ich die PAramter andere Modelle trainieren soll (wäre random).

An sich steht ja alles hier in der Mappe, vllt könnt ihr ja noch paar Regression nach verschiedener Paramter durchführen (Nach Trinker, BMI o.ä.)

Wenn Sie sowohl modellieren als auch prädizieren wollen, führen Sie die Modellierung nur auf den Trainingsdaten durch, um zu vermeiden, dass Sie ungewollt aus Ihren Testdaten lernen. Trainieren Sie das Prädiktionsmodell im zweiten Schritt aufgrund der Erkenntnisse der Modellierung.